In [33]:
import cv2
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import cv2

In [5]:
# Initialize image data generator with rescaling
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

# Preprocess all test images
train_generator = train_data_gen.flow_from_directory(
        'data/train',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

# Preprocess all train images
validation_generator = validation_data_gen.flow_from_directory(
        'data/test',
        target_size=(48, 48),
        batch_size=64,
        color_mode="grayscale",
        class_mode='categorical')

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [11]:
# create model structure
emotion_model = Sequential()

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(48, 48, 1)))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

In [15]:
# Train the neural network/model
emotion_model_info = emotion_model.fit(
        train_generator,
        steps_per_epoch=28709 // 64,
        epochs=20,        validation_data=validation_generator,
        validation_steps=7178 // 64)


Epoch 1/20
448/448 [==============================] - 425s 949ms/step - loss: 0.9293 - accuracy: 0.6559 - val_loss: 1.0813 - val_accuracy: 0.5979
Epoch 2/20
448/448 [==============================] - 163s 363ms/step - loss: 0.9083 - accuracy: 0.6626 - val_loss: 1.0818 - val_accuracy: 0.6032
Epoch 3/20
448/448 [==============================] - 174s 388ms/step - loss: 0.8829 - accuracy: 0.6746 - val_loss: 1.0719 - val_accuracy: 0.6031
Epoch 4/20
448/448 [==============================] - 181s 404ms/step - loss: 0.8634 - accuracy: 0.6806 - val_loss: 1.0642 - val_accuracy: 0.6062
Epoch 5/20
448/448 [==============================] - 190s 424ms/step - loss: 0.8389 - accuracy: 0.6905 - val_loss: 1.0797 - val_accuracy: 0.6051
Epoch 6/20
448/448 [==============================] - 177s 395ms/step - loss: 0.8150 - accuracy: 0.6979 - val_loss: 1.0610 - val_accuracy: 0.6131
Epoch 7/20
448/448 [==============================] - 167s 372ms/step - loss: 0.7910 - accuracy: 0.7110 - val_loss: 1.0653 -

In [17]:
model_json = emotion_model.to_json()
with open("models/emotion_model.json", "w") as json_file:
    json_file.write(model_json)

# save trained model weight in .h5 file
emotion_model.save_weights('models/emotion_model.h5')

In [52]:
img_check=cv2.imread('data\train\angry\Training_3908.jpg')
cv2.imshow(img_check)
cv2.waitkey()

AttributeError: module 'cv2' has no attribute 'waitkey'

In [34]:
img=cv2.imread('grayed.jpg')

In [38]:
image_tensor = tf.convert_to_tensor(img, dtype=tf.float32)

# Add dimension to match with input mode 
image_tensor = tf.expand_dims(image_tensor, 0)

In [42]:
image_tensor

<tf.Tensor: shape=(1, 48, 48, 3), dtype=float32, numpy=
array([[[[0.27450982, 0.27450982, 0.27450982],
         [0.1764706 , 0.1764706 , 0.1764706 ],
         [0.22352941, 0.22352941, 0.22352941],
         ...,
         [0.45490196, 0.45490196, 0.45490196],
         [0.5294118 , 0.5294118 , 0.5294118 ],
         [0.49803922, 0.49803922, 0.49803922]],

        [[0.14509805, 0.14509805, 0.14509805],
         [0.18039216, 0.18039216, 0.18039216],
         [0.20392157, 0.20392157, 0.20392157],
         ...,
         [0.5647059 , 0.5647059 , 0.5647059 ],
         [0.3137255 , 0.3137255 , 0.3137255 ],
         [0.5176471 , 0.5176471 , 0.5176471 ]],

        [[0.29803923, 0.29803923, 0.29803923],
         [0.14901961, 0.14901961, 0.14901961],
         [0.18039216, 0.18039216, 0.18039216],
         ...,
         [0.2784314 , 0.2784314 , 0.2784314 ],
         [0.36862746, 0.36862746, 0.36862746],
         [0.58431375, 0.58431375, 0.58431375]],

        ...,

        [[0.23137255, 0.23137255, 0.

In [41]:
image_tensor=image_tensor/255

In [43]:
emotion_model.predict(image_tensor)

ValueError: in user code:

    File "C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2341, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2327, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2315, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\training.py", line 2283, in predict_step
        return self(x, training=False)
    File "C:\Users\HP\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\HP\anaconda3\Lib\site-packages\keras\src\engine\input_spec.py", line 280, in assert_input_compatibility
        raise ValueError(

    ValueErro